# Packages

In [2]:
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd
import numpy as np
import polars as pl
import scipy.stats as stats
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import math

from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from statsmodels.graphics.tsaplots import plot_pacf

from sklearn.model_selection import GroupKFold, KFold, train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, make_scorer, accuracy_score, median_absolute_error, classification_report, mean_absolute_error, roc_auc_score, roc_curve
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from imblearn.over_sampling import RandomOverSampler
from scipy.optimize import minimize

import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor, RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression

import optuna

import requests
import re
import tabula
import pdfplumber

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: "%.4f" % x)
# pd.options.plotting.backend = "plotly"

plt.style.use('ggplot')
sns.set_style('darkgrid')

# Reading Data

In [ ]:
dir_name = "/Users/kylewang/Documents/yale/SDS4250_StatCaseStudies/OlympicFigureSkating/OWG10_ScoresCSV_SOV"
file_name = "owg10_Ladies_FS_Scores_SOV.csv"
full_path = dir_name + "/" + file_name

df = pd.read_csv(full_path)
df.head()

,rank,name,country,starting_number,total_segment_score,total_element_score,total_component_score,deductions,type,category,base_value,highlight,goe,factor,j1,j2,j3,j4,j5,j6,j7,j8,j9,scores_of_panel
0,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,3Lz+3T,10.0000,0.0000,2.0000,NaN,2.0000,2.0000,2.0000,3.0000,2.0000,2.0000,0.0000,2.0000,2.0000,12.0000
1,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,3F,5.5000,0.0000,1.8000,NaN,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,1.0000,1.0000,2.0000,7.3000
2,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,2A+2T+2Lo,6.3000,0.0000,1.4000,NaN,1.0000,1.0000,1.0000,2.0000,1.0000,2.0000,2.0000,2.0000,1.0000,7.7000
3,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,FCoSp4,3.0000,0.0000,0.8000,NaN,0.5000,0.5000,0.5000,1.0000,0.5000,0.5000,1.0000,1.0000,1.0000,3.8000
4,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,SpSq4,3.4000,0.0000,2.0000,NaN,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,3.0000,2.0000,2.0000,5.4000


In [4]:
df.shape

(432, 24)

In [5]:
display(df)

,rank,name,country,starting_number,total_segment_score,total_element_score,total_component_score,deductions,type,category,base_value,highlight,goe,factor,j1,j2,j3,j4,j5,j6,j7,j8,j9,scores_of_panel
0,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,3Lz+3T,10.0000,0.0000,2.0000,NaN,2.0000,2.0000,2.0000,3.0000,2.0000,2.0000,0.0000,2.0000,2.0000,12.0000
1,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,3F,5.5000,0.0000,1.8000,NaN,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,1.0000,1.0000,2.0000,7.3000
2,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,2A+2T+2Lo,6.3000,0.0000,1.4000,NaN,1.0000,1.0000,1.0000,2.0000,1.0000,2.0000,2.0000,2.0000,1.0000,7.7000
3,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,FCoSp4,3.0000,0.0000,0.8000,NaN,0.5000,0.5000,0.5000,1.0000,0.5000,0.5000,1.0000,1.0000,1.0000,3.8000
4,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,SpSq4,3.4000,0.0000,2.0000,NaN,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,3.0000,2.0000,2.0000,5.4000
5,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,2A+3T,7.5000,0.0000,2.0000,NaN,2.0000,2.0000,2.0000,3.0000,2.0000,2.0000,2.0000,2.0000,2.0000,9.5000
6,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,3S,4.9500,0.0000,1.4000,NaN,1.0000,0.0000,1.0000,2.0000,1.0000,1.0000,1.0000,2.0000,2.0000,6.3500
7,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,3Lz,6.6000,0.0000,2.0000,NaN,2.0000,3.0000,2.0000,3.0000,2.0000,2.0000,2.0000,1.0000,2.0000,8.6000
8,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,SlSt3,3.3000,0.0000,1.0000,NaN,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.5000,1.0000,1.0000,4.3000
9,1,KIM Yu-Na,KOR,21,150.0600,78.3000,71.7600,NaN,ee,2A,3.8500,0.0000,1.4000,NaN,1.0000,1.0000,1.0000,2.0000,2.0000,1.0000,2.0000,1.0000,1.0000,5.2500
